## Let's try to predict titanic survivors with basic deep learning

### First: Import and clean dataset:


In [159]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

input_file = "train.csv"
df = pd.read_csv(input_file, header = 0)

df = df.dropna() 

df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
21,22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0000,D56,S
23,24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5000,A6,S
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0000,C23 C25 C27,S
52,53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C
54,55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C


### Remove unnecesary data

In [160]:
df = df.drop('Name', 1)
df = df.drop('Ticket', 1)
df = df.drop('Cabin', 1)
df = df.drop('PassengerId', 1)

df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,1,female,38.0,1,0,71.2833,C
3,1,1,female,35.0,1,0,53.1000,S
6,0,1,male,54.0,0,0,51.8625,S
10,1,3,female,4.0,1,1,16.7000,S
11,1,1,female,58.0,0,0,26.5500,S
21,1,2,male,34.0,0,0,13.0000,S
23,1,1,male,28.0,0,0,35.5000,S
27,0,1,male,19.0,3,2,263.0000,S
52,1,1,female,49.0,1,0,76.7292,C
54,0,1,male,65.0,0,1,61.9792,C


### Define X and y

In [161]:
X = df.drop('Survived', axis = 1)
y = df[['Survived']]

X.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,female,38.0,1,0,71.2833,C
3,1,female,35.0,1,0,53.1000,S
6,1,male,54.0,0,0,51.8625,S
10,3,female,4.0,1,1,16.7000,S
11,1,female,58.0,0,0,26.5500,S
21,2,male,34.0,0,0,13.0000,S
23,1,male,28.0,0,0,35.5000,S
27,1,male,19.0,3,2,263.0000,S
52,1,female,49.0,1,0,76.7292,C
54,1,male,65.0,0,1,61.9792,C


### Encode categorical variables:

In [162]:
colT = ColumnTransformer(
   [("dummy_col", OneHotEncoder(categories=[['male', 'female'],
                                            ['C', 'S', 'Q']]), [1,6])
   ], remainder="passthrough")


X = colT.fit_transform(X)
X[:10]

array([[  0.    ,   1.    ,   1.    ,   0.    ,   0.    ,   1.    ,
         38.    ,   1.    ,   0.    ,  71.2833],
       [  0.    ,   1.    ,   0.    ,   1.    ,   0.    ,   1.    ,
         35.    ,   1.    ,   0.    ,  53.1   ],
       [  1.    ,   0.    ,   0.    ,   1.    ,   0.    ,   1.    ,
         54.    ,   0.    ,   0.    ,  51.8625],
       [  0.    ,   1.    ,   0.    ,   1.    ,   0.    ,   3.    ,
          4.    ,   1.    ,   1.    ,  16.7   ],
       [  0.    ,   1.    ,   0.    ,   1.    ,   0.    ,   1.    ,
         58.    ,   0.    ,   0.    ,  26.55  ],
       [  1.    ,   0.    ,   0.    ,   1.    ,   0.    ,   2.    ,
         34.    ,   0.    ,   0.    ,  13.    ],
       [  1.    ,   0.    ,   0.    ,   1.    ,   0.    ,   1.    ,
         28.    ,   0.    ,   0.    ,  35.5   ],
       [  1.    ,   0.    ,   0.    ,   1.    ,   0.    ,   1.    ,
         19.    ,   3.    ,   2.    , 263.    ],
       [  0.    ,   1.    ,   1.    ,   0.    ,   0.    ,   1.  

### Remove dummy variables
Column nro 1 and 2 (0 is for sex and 2 for harvour)

In [163]:
X = X [:, [1,3,4,5,6,7,8,9]]
X[:3]

array([[ 1.    ,  0.    ,  0.    ,  1.    , 38.    ,  1.    ,  0.    ,
        71.2833],
       [ 1.    ,  1.    ,  0.    ,  1.    , 35.    ,  1.    ,  0.    ,
        53.1   ],
       [ 0.    ,  1.    ,  0.    ,  1.    , 54.    ,  0.    ,  0.    ,
        51.8625]])

In [164]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X)
X_train

array([[ 1.03901177, -1.31580518, -0.10511767, ...,  0.83362754,
        -0.63172982, -0.0971798 ],
       [ 1.03901177,  0.75999093, -0.10511767, ...,  0.83362754,
        -0.63172982, -0.3359971 ],
       [-0.96245301,  0.75999093, -0.10511767, ..., -0.7230443 ,
        -0.63172982, -0.35225028],
       ...,
       [ 1.03901177, -1.31580518, -0.10511767, ..., -0.7230443 ,
         0.69708118,  0.05878503],
       [ 1.03901177,  0.75999093, -0.10511767, ..., -0.7230443 ,
        -0.63172982, -0.63938976],
       [-0.96245301, -1.31580518, -0.10511767, ..., -0.7230443 ,
        -0.63172982, -0.63938976]])

In [167]:
import keras

from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu', input_dim = 8))

# Adding the second hidden layer
classifier.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y, batch_size = 10, epochs = 100)
 

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
183/183 [==============================] - 1s 7ms/step - loss: 0.6919 - acc: 0.6557
Epoch 2/100
183/183 [==============================] - 0s 233us/step - loss: 0.6893 - acc: 0.6721
Epoch 3/100
183/183 [==============================] - 0s 219us/step - loss: 0.6866 - acc: 0.6721
Epoch 4/100
183/183 [==============================] - 0s 217us/step - loss: 0.6831 - acc: 0.6721
Epoch 5/100
183/183 [==============================] - 0s 228us/step - loss: 0.6790 - acc: 0.6721
Epoch 6/100
183/183 [==============================] - 0s 225us/step - loss: 0.6735 - acc: 0.6721
Epoch 7/100
183/183 [==============================] - 0s 227us/step - loss: 0.6665 - acc: 0.6721
Epoch 8/100
183/183 [==============================] - 0s 220us/step - loss: 0.6574 - acc: 0.6721
Epoch 9/100
183/183 [==============================] - 0s 223us/step - loss: 0.6442 - acc: 0.6721
Epoch 10/100
183/183 [==============================] - 0s 222us/step - 

183/183 [==============================] - 0s 250us/step - loss: 0.4122 - acc: 0.7869
Epoch 82/100
183/183 [==============================] - 0s 258us/step - loss: 0.4119 - acc: 0.7978
Epoch 83/100
183/183 [==============================] - 0s 261us/step - loss: 0.4114 - acc: 0.7978
Epoch 84/100
183/183 [==============================] - 0s 264us/step - loss: 0.4109 - acc: 0.7978
Epoch 85/100
183/183 [==============================] - 0s 268us/step - loss: 0.4114 - acc: 0.7923
Epoch 86/100
183/183 [==============================] - 0s 264us/step - loss: 0.4110 - acc: 0.7978
Epoch 87/100
183/183 [==============================] - 0s 261us/step - loss: 0.4107 - acc: 0.7978
Epoch 88/100
183/183 [==============================] - 0s 266us/step - loss: 0.4104 - acc: 0.7923
Epoch 89/100
183/183 [==============================] - 0s 261us/step - loss: 0.4101 - acc: 0.7978
Epoch 90/100
183/183 [==============================] - 0s 253us/step - loss: 0.4104 - acc: 0.7978
Epoch 91/100
183/183 [=

## Prepare training set

In [173]:
input_test_file = "test.csv"
df_test = pd.read_csv(input_file, header = 0)
df_test = df_test.dropna() 

df_test = df_test.drop('Name', 1)
df_test = df_test.drop('Ticket', 1)
df_test = df_test.drop('Cabin', 1)
df_test = df_test.drop('PassengerId', 1)

X_test = df_test.drop('Survived', axis = 1)
y_test = df_test[['Survived']]

X_test = colT.fit_transform(X_test)
X_test = X_test [:, [1,3,4,5,6,7,8,9]]

X_test_train = sc.fit_transform(X_test)

y_pred = classifier.predict(X_test_train)
y_pred = (y_pred > 0.5)


In [175]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[ 43,  17],
       [ 19, 104]])

## Accuracy

In [183]:
accuracy = (cm[0][0] + cm[1][1]) / y_test.size
accuracy

0.8032786885245902